# Access CCMP data on Pangeo

This tutorial shows how to access the CCMP wind speed data, makes a few plots of the data, and then compares it to a buoy wind speed.

## [More info](https://podaac.jpl.nasa.gov/MEaSUREs-CCMP?sections=about)

The Cross-Calibrated Multi-Platform (CCMP) Ocean Surface Wind Vector Analyses is part of the NASA Making Earth System Data Records for Use in Research Environments (MEaSUREs) Program. MEaSUREs, develops consistent global- and continental-scale Earth System Data Records by supporting projects that produce data using proven algorithms and input.

CCMP (Atlas et al., 2011) provides a consistent, gap-free long-term time-series of ocean surface wind vector analysis fields from July 1987 through June 2011. The CCMP datasets combine cross-calibrated satellite winds using a Variational Analysis Method (VAM) to produce a high-resolution (0.25 degree) gridded analysis.

Reference: Atlas, R., R. N. Hoffman, J. Ardizzone, S. M. Leidner, J. C. Jusem, D. K. Smith, D. Gombos, 2011: A cross-calibrated, multiplatform ocean surface wind velocity product for meteorological and oceanographic applications. Bull. Amer. Meteor. Soc., 92, 157-174. doi: 10.1175/2010BAMS2946.1

Data is NRT from 4/1/2019 - present.
The 6-hourly VAM analyses are centered at 0,6,12 and 18z

## Credit:
- [Chelle Gentemann](mailto:cgentemann@faralloninstitute.org), [Farallon Institute](http://www.faralloninstitute.org/), [Twitter](https://twitter.com/ChelleGentemann) - creation of Zarr data store and tutorial
- [Charles Blackmon Luca](mailto:blackmon@ldeo.columbia.edu), [LDEO](https://www.ldeo.columbia.edu/) - help with moving to Pangeo storage and intake update
- [Willi Rath](mailto:wrath@geomar.de), [GEOMAR](https://www.geomar.de/en/), [Twitter](https://twitter.com/RathWilli) - motivated CG to move data to Pangeo!

In [ ]:
#libs for reading data
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import intake
import dask

#libs for dask gateway
from dask_gateway import Gateway
from dask.distributed import Client

### Start a cluster, a group of computers that will work together.

(A cluster is the key to big data analysis on on Cloud.)

- This will set up a [dask kubernetes](https://docs.dask.org/en/latest/setup/kubernetes.html) cluster for your analysis and give you a path that you can paste into the top of the Dask dashboard to visualize parts of your cluster.  
- You don't need to paste the link below into the Dask dashboard for this to work, but it will help you visualize progress.
- Try 20 workers to start (during the tutorial) but you can increase to speed things up later

In [ ]:
gateway = Gateway()
cluster = gateway.new_cluster()
#cluster.adapt(minimum=1, maximum=200)
cluster.scale(200)
client = Client(cluster)
cluster

** ☝️ Don’t forget to click the link above or copy it to the Dask dashboard ![images.png](attachment:images.png) on the left to view the scheduler dashboard! **

### Initialize Dataset

Here we load the dataset from the zarr store. Note that this very large dataset (273 GB) initializes nearly instantly, and we can see the full list of variables and coordinates.

### Examine Metadata

For those unfamiliar with this dataset, the variable metadata is very helpful for understanding what the variables actually represent
Printing the dataset will show you the dimensions, coordinates, and data variables with clickable icons at the end that show more metadata and size.

In [ ]:
%%time
cat_pangeo = intake.open_catalog("https://raw.githubusercontent.com/pangeo-data/pangeo-datastore/master/intake-catalogs/master.yaml")

ds_ccmp = cat_pangeo.atmosphere.nasa_ccmp_wind_vectors.to_dask()

#calculate wind speed and add attributes to new variable
ds_ccmp['wspd'] = np.sqrt(ds_ccmp.uwnd**2 + ds_ccmp.vwnd**2)
ds_ccmp.wspd.attrs=ds_ccmp.uwnd.attrs
ds_ccmp.wspd.attrs['long_name']='wind speed at 10 meters'

ds_ccmp

# Plot a timeseries of the average wind speed over a region

- use [.sel](http://xarray.pydata.org/en/stable/generated/xarray.DataArray.sel.html#xarray-dataarray-sel) and slice to select a region of data
- use [.mean](http://xarray.pydata.org/en/stable/generated/xarray.DataArray.mean.html#xarray.DataArray.mean) to calculate the spatial mean
- use [.plot](http://xarray.pydata.org/en/stable/generated/xarray.DataArray.plot.html#xarray-dataarray-plot) to plot a timeseries of the mean wind speed

In [ ]:
%%time

subset = ds_ccmp.sel(latitude=slice(30,50),longitude=slice(200,210))

ts = subset.mean({'latitude','longitude'},keep_attrs=True)

ts.wspd.plot()


# Plot a map of the annual average wind speed

In [ ]:
%%time

ds_ccmp.sel(time=slice('2000-01-01','2000-12-31')).mean({'time'}).wspd.plot()

# Make a Hovmoller type plot

In [ ]:
%%time

ds_ccmp.sel(latitude=0.125,longitude=slice(120,275)).wspd.plot(vmin=0,vmax=12,cmap='magma')

# Compare to buoy data
Data from NDBC buoys, which measure wind speed are [here](https://dods.ndbc.noaa.gov/) and can be read via an THREDDS server.
- read in NDBC buoy data
- find closest CCMP data and linearly interpolate to buoy location
- examine a timeseries, caluclate bias and STD

In [ ]:
url='https://dods.ndbc.noaa.gov/thredds/dodsC/data/cwind/51003/51003.ncml'
ds = xr.open_dataset(url)
# The longitude is on a -180 to 180, CCMP is 0-360, so make sure to convert
ds.coords['longitude'] = np.mod(ds['longitude'], 360)
ds

In [ ]:
time_start = np.max([ds_ccmp.time[0].data,ds.time[0].data])
time_stop = np.min([ds_ccmp.time[-1].data,ds.time[-1].data])
#cut off time before / after buoy obs, linearly interpol data for buoy location
ccmp_buoy = ds_ccmp.sel(time=slice(time_start,time_stop)).interp(latitude=ds.latitude,longitude=ds.longitude,method='linear')
ds = ds.sel(time=slice(time_start,time_stop))


In [ ]:
%%time
# go from 30 min to 6-hourly sampling, resample with mean for +- 3 hours centered on timestamp
#ds = ds.resample(time='6H',loffset='180min',base=3).mean()  
# OMG!!! resample is sooo slow.  doing this cludge instead, same to within 6 significant digits
ds = ds.rolling(time=36,center=True).mean() 
ds = ds.interp(time=ccmp_buoy.time,method='nearest')
#make sure time span matches
ds_buoy = ds
ds_buoy

In [ ]:
ccmp_buoy.wspd.plot(color='r')
ds_buoy.wind_spd.plot(color='b')

In [ ]:
# let's remove the seasonal cycle so we can compare the data more clearly
ts_ccmp_climatology = ccmp_buoy.groupby('time.dayofyear').mean('time',keep_attrs=True,skipna=True)
ts_ccmp_anomaly = ccmp_buoy.groupby('time.dayofyear')-ts_ccmp_climatology
ts_buoy_climatology = ds_buoy.groupby('time.dayofyear').mean('time',keep_attrs=True,skipna=True)
ts_buoy_anomaly = ds_buoy.groupby('time.dayofyear')-ts_buoy_climatology

In [ ]:
(ts_ccmp_anomaly.wspd-ts_buoy_anomaly.wind_spd).plot()

In [ ]:
print('buoy minus satellite wind speeds')
rdif = (ts_buoy_anomaly.wind_spd-ts_ccmp_anomaly.wspd).load()
print('mean:',rdif.mean().data)
print('std:',rdif.std().data)